In [1]:
import numpy as np 
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import heapq

In [2]:
# 'essentials' implies essential facial skincare products
df = pd.read_csv('result.csv')

In [3]:
df.head()

,label,url,brand,name,price,skin type,spf,concern,concern 2,concern 3,key ingredient,formulation
0,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,Lakme,Absolute Perfect Radiance Skin Lightening Day ...,₹ 79,All,NaN,General Care,NaN,NaN,NaN,Cream
1,face-moisturisers,https://www.myntra.com/face-moisturisers/bioti...,Biotique,Bio Morning Nectar Flawless Sustainable Skin M...,₹ 165,All,NaN,Uneven Skin Tone,Hydration,Dark Spots,Honey,Lotion
2,face-moisturisers,https://www.myntra.com/face-moisturisers/nivea...,Nivea,Unisex Aloe Hydration Skin Cream 100 ml,₹ 92,All,NaN,Dull Skin,Dryness,General Care,Aloe Vera,Cream
3,face-moisturisers,https://www.myntra.com/face-moisturisers/vi-jo...,VI-JOHN,Women Set of 5 Saffron Fairness Cream Advanced,₹ 187,All,15 to 30,Softening and Smoothening,Softening and Smoothening,Sun Protection,Vitamin E,Cream
4,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,Lakme,Peach Milk Soft Creme 150g,₹ 192,All,NaN,General Care,NaN,NaN,NaN,Cream


In [4]:
df['label'].value_counts()

label
face-moisturisers    300
cleanser             300
concealer            300
mask-and-peel        300
foundation           300
sunscreen            272
eye-cream            100
Name: count, dtype: int64

In [5]:
df.isna().sum()

label                0
url                  0
brand               46
name                46
price               46
skin type          309
spf               1452
concern            380
concern 2         1136
concern 3         1319
key ingredient    1159
formulation        384
dtype: int64

In [6]:
df['concern 2'].fillna('', inplace = True)
df['concern 3'].fillna('', inplace = True)
df['concern'] = df['concern'] + ',' + df['concern 2'] + ',' + df['concern 3']

In [7]:
df['concern']

0                                          General Care,,
1                   Uneven Skin Tone,Hydration,Dark Spots
2                          Dull Skin,Dryness,General Care
3       Softening and Smoothening,Softening and Smooth...
4                                          General Care,,
                              ...                        
1867                                       Dark Circles,,
1868                Fine Lines and Wrinkles,Dark Circles,
1869                   Dark Circles,Dark Circles,Eye Bags
1870                                                  NaN
1871                        Dark Circles,Eye Bags,Dryness
Name: concern, Length: 1872, dtype: object

In [8]:
df.drop(columns=['concern 2', 'concern 3', 'spf', 'key ingredient', 'formulation'], inplace = True)
df['concern'].value_counts()

concern
Sun Protection,,                                       202
Dark Spots,,                                            81
General Care,,                                          68
Dark Circles,,                                          51
Waterproof,,                                            43
                                                      ... 
Acne or Blemishes,Pigmentation,Anti-Pollution            1
Lightening,Dark Spots,                                   1
Daily Use,Acne or Blemishes,Fine Lines and Wrinkles      1
Anti-Pollution,Acne or Blemishes,Excess Oil              1
Lightening,Dryness,                                      1
Name: count, Length: 438, dtype: int64

In [9]:
df2 = df[((df['label'] == 'face-moisturisers') | (df['label'] == 'mask-and-peel') | (df['label'] == 'cleanser') | (df['label'] == 'eye-cream'))]
df2
LABELS = list(df2.label.unique())

In [10]:
df2 = df2[df2['skin type'].isna() == False]
df2.index = [i for i in range(0, len(df2))]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 967 entries, 0 to 966
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      967 non-null    object
 1   url        967 non-null    object
 2   brand      967 non-null    object
 3   name       967 non-null    object
 4   price      967 non-null    object
 5   skin type  967 non-null    object
 6   concern    939 non-null    object
dtypes: object(7)
memory usage: 60.4+ KB


In [11]:
df2[df2['concern'].isna() == True]['label'].value_counts()

label
face-moisturisers    10
mask-and-peel         9
cleanser              5
eye-cream             4
Name: count, dtype: int64

In [12]:
df2[df2['label'] ==  'eye-cream']['concern'].value_counts()

concern
Fine Lines and Wrinkles,Dark Circles,Eye Bags    16
Dark Circles,Eye Bags,                           12
Dark Circles,,                                   11
Fine Lines and Wrinkles,Dark Circles,             9
Dark Circles,Dark Circles,Eye Bags                8
Dark Circles,Eye Bags,Dryness                     7
Dark Circles,Eye Bags,Dark Circles                6
Dryness,Dark Circles,                             3
Dark Circles,Dryness,                             3
Fine Lines and Wrinkles,Dryness,                  3
Fine Lines and Wrinkles,,                         3
Dryness,,                                         2
Dryness,Eye Bags,Dark Circles                     2
Fine Lines and Wrinkles,Dark Circles,Dryness      2
Lightening,Dark Circles,Eye Bags                  2
Fine Lines and Wrinkles,Dryness,Eye Bags          1
Dark Circles,Dark Circles,                        1
Dark Circles,Dryness,Eye Bags                     1
Lightening,,                                      1
Ligh

In [13]:
df2['concern'] = df2['concern'].str.lower()
# df2[df2['label'] == 'face-moisturisers']['concern'].value_counts()
# df2[df2['label'] == 'mask-and-peel']['concern'].value_counts()
df2[df2['label'] == 'sunscreen']['concern'].value_counts()

Series([], Name: count, dtype: int64)

In [14]:
top_concerns = {'face-moisturisers':'general care', 'mask-and-peel':'daily use', 'cleanser':'general care', 'eye-cream':'fine lines,wrinkles,dark circles,eye bags' }
entries = len(df2)
for i in range(entries):
    label = df2.iloc[i]['label']
    if pd.isnull(df2.iloc[i]['concern']):
        df2.iloc[i]['concern'] = top_concerns[label]
df2.info()


C:\Users\Akshaya\AppData\Local\Temp\ipykernel_8352\49380362.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.iloc[i]['concern'] = top_concerns[label]


<class 'pandas.core.frame.DataFrame'>
Index: 967 entries, 0 to 966
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      967 non-null    object
 1   url        967 non-null    object
 2   brand      967 non-null    object
 3   name       967 non-null    object
 4   price      967 non-null    object
 5   skin type  967 non-null    object
 6   concern    939 non-null    object
dtypes: object(7)
memory usage: 60.4+ KB


In [15]:
# df2.to_csv('general_skincare.csv')

In [16]:
df2['brand'] = df2['brand'].str.lower()
df2['name'] = df2['name'].str.lower()
df2['skin type'] = df2['skin type'].str.lower()
df2['concern'] = df2['concern'].str.replace(' and ', ',').str.replace(' or ', ',')
df2

,label,url,brand,name,price,skin type,concern
0,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,lakme,absolute perfect radiance skin lightening day ...,₹ 79,all,"general care,,"
1,face-moisturisers,https://www.myntra.com/face-moisturisers/bioti...,biotique,bio morning nectar flawless sustainable skin m...,₹ 165,all,"uneven skin tone,hydration,dark spots"
2,face-moisturisers,https://www.myntra.com/face-moisturisers/nivea...,nivea,unisex aloe hydration skin cream 100 ml,₹ 92,all,"dull skin,dryness,general care"
3,face-moisturisers,https://www.myntra.com/face-moisturisers/vi-jo...,vi-john,women set of 5 saffron fairness cream advanced,₹ 187,all,"softening,smoothening,softening,smoothening,su..."
4,face-moisturisers,https://www.myntra.com/face-moisturisers/lakme...,lakme,peach milk soft creme 150g,₹ 192,all,"general care,,"
...,...,...,...,...,...,...,...
962,eye-cream,https://www.myntra.com/eye-cream/skinkraft/ski...,skinkraft,vitamin c+e under eye gel for pigmented under ...,₹ 449,all,"dark circles,,"
963,eye-cream,https://www.myntra.com/eye-cream/casmara/casma...,casmara,eye-cream 15 ml,₹ 2200,all,"fine lines,wrinkles,dark circles,"
964,eye-cream,https://www.myntra.com/eye-cream/myglamm/mygla...,myglamm,white youthfull hydrating eye cream with water...,₹ 716,all,"dark circles,dark circles,eye bags"
965,eye-cream,https://www.myntra.com/eye-cream/makeup-revolu...,makeup revolution london,hydro bank hydrating & cooling eye balm,₹ 1050,all,NaN


In [17]:
def concern_elements(comma_sep_concerns):
    words = comma_sep_concerns.split(',')
    for w in words:
        if w != '':
            temp = w.rstrip()
            if temp in concerns:
                concerns[temp] += 1
            else:
                concerns[temp] = 1


            
# features
list(df2['skin type'].unique())
concerns = {}
for i in range(entries):
    concern_elements(df2.iloc[i]['concern'])

AttributeError: 'float' object has no attribute 'split'

In [18]:
print(sorted(concerns.items(), key =
             lambda kv:(kv[1], kv[0])))   

[('irregular textures', 1), ('skin inflammation', 1), ('blackheads', 2), ('skin sagging', 2), ('sun protection', 2), ('whiteheads', 2), ('acne', 3), ('blemishes', 3), ('excess oil', 3), ('deep nourishment', 4), ('dark spots', 5), ('fine lines', 5), ('wrinkles', 5), ('dull skin', 6), ('uneven skin tone', 6), ('pigmentation', 7), ('smoothening', 7), ('softening', 7), ('dryness', 9), ('hydration', 9), ('general care', 12)]


In [19]:
concerns.pop('anti acne scarring')
concerns.pop('anti-ageing')
concerns.pop('skin tightening')
concerns.pop('blackheads removal')
concerns.pop('irregular textures')
concerns.pop('skin inflammation')
concerns.pop('oil control')
concerns.pop('redness')
concerns.pop('oily skin')
concerns.pop('skin sagging')
concerns.pop('sun protection')
concerns.pop('lightening')
concerns.pop('tan removal')
concerns.pop('excess oil')

KeyError: 'anti acne scarring'

In [20]:
features = list(df2['skin type'].unique()) + list(concerns)
(features)

features = ['normal','dry','oily','combination', 'acne','sensitive','fine lines', 'wrinkles', 'redness', 'dull', 'pore', 'pigmentation', 'blackheads','whiteheads', 'blemishes', 'dark circles', 'eye bags', 'dark spots']

In [21]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 967 entries, 0 to 966
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      967 non-null    object
 1   url        967 non-null    object
 2   brand      967 non-null    object
 3   name       967 non-null    object
 4   price      967 non-null    object
 5   skin type  967 non-null    object
 6   concern    939 non-null    object
dtypes: object(7)
memory usage: 60.4+ KB


In [22]:
len(features)

18

In [23]:
def search_concern(target, i):
    if target in df2.iloc[i]['concern']:
        return True
    return False

one_hot_encodings = np.zeros([entries, len(features)])

#skin types first
for i in range(entries):
    for j in range(5):
        target = features[j]
        sk_type = df2.iloc[i]['skin type']
        if sk_type == 'all':
            one_hot_encodings[i][0:5] = 1
        elif target == sk_type:
            one_hot_encodings[i][j] = 1

#other features
for i in range(entries):
    for j in range(5, len(features)):
        feature = features[j]
        if feature in df2.iloc[i]['concern']:
            one_hot_encodings[i][j] = 1
            
    

TypeError: argument of type 'float' is not iterable

In [24]:
x = one_hot_encodings[456]

In [25]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(one_hot_encodings)
distances, indices = nbrs.kneighbors(one_hot_encodings)

# get_index_from_name('peach milk soft creme 150g')

In [26]:
# finding the closes data points
def recs_nn(query=None,id=None):
    if id:
        for id in indices[id][1:]:
            print(anime.ix[id]["name"])
    if query:
        found_id = get_index_from_name(query)
        for id in indices[found_id][1:]:
            print(index2prod(id))

In [27]:
# utility functions
def name2index(name):
    return df2[df2["name"]==name].index.tolist()[0]

def index2prod(index):
    return df2.iloc[index]

In [28]:
def wrap(info_arr):
    result = {}
#     print(info_arr)
    result['brand'] = info_arr[0]
    result['name'] = info_arr[1]
    result['price'] = info_arr[2]
    result['url'] = info_arr[3]
    result['skin type'] = info_arr[4]
    result['concern'] = str(info_arr[5]).split(',')
    return result

# recommend top 10 similar items from a category
def recs_cs(vector = None, name = None, label = None, count = 5):
    products = []
    if name:
        idx = name2index(name)
        fv = one_hot_encodings[idx]
    elif vector:
        fv = vector
    cs_values = cosine_similarity(np.array([fv, ]), one_hot_encodings)
    df2['cs'] = cs_values[0]
    
    if label:
        dff = df2[df2['label'] == label]
    else:
        dff = df2
    
    if name:
        dff = dff[dff['name'] != name]
    recommendations = dff.sort_values('cs', ascending=False).head(count)
    #   print(f"Top {count} matching {label} items")
    data = recommendations[['brand', 'name', 'price', 'url','skin type','concern']].to_dict('split')['data']
    for element in data:
        products.append(wrap(element))
    return products

In [29]:
# overall recommendation
def recs_essentials(vector = None, name = None):
#     print("ESSENTIALS:")
    response = {}
    for label in LABELS:
#         print(f"{label}:")
        if name: 
            r = recs_cs(None, name, label)
        elif vector:
            r = recs_cs(vector, None, label)
        response[label] = r
    return response

In [30]:
# features = ['normal','dry','oily','combination', 'acne','sensitive','fine lines', 'wrinkles', 'redness', 'dull', 'pore ', 'pigmentation', 'blackheads','whiteheads', 'blemishes', 'dark circles', 'eye bags', 'dark spots']
x = [1,1,1,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0]

y = recs_essentials(x, None)
y

{'face-moisturisers': [{'brand': 'lotus herbals',
   'name': 'sustainable professional phyto-rx whitening brightening night creme 50g',
   'price': '₹ 419',
   'url': 'https://www.myntra.com/face-moisturisers/lotus-herbals/lotus-herbals-sustainable-professional-phyto-rx-whitening--brightening-night-creme-50g/10365395/buy',
   'skin type': 'all',
   'concern': ['dull skin', '', '']},
  {'brand': 'nivea',
   'name': 'unisex aloe hydration skin cream 100 ml',
   'price': '₹ 92',
   'url': 'https://www.myntra.com/face-moisturisers/nivea/nivea-unisex-aloe-hydration-skin-cream-100-ml/8529167/buy',
   'skin type': 'all',
   'concern': ['dull skin', 'dryness', 'general care']},
  {'brand': 'the face shop',
   'name': 'unisex rice & ceramide moisturizing emulsion 150 ml',
   'price': '₹ 712',
   'url': 'https://www.myntra.com/face-moisturisers/the-face-shop/the-face-shop-unisex-rice--ceramide-moisturizing-emulsion-150-ml/9113689/buy',
   'skin type': 'all',
   'concern': ['dryness', 'dull skin'